In [ ]:
!pip3 install adapter-transformers

In [5]:
import torch, os
from torch import cuda
import numpy as np
# transformers
from adapters import AdapterTrainer
from transformers import (BertTokenizerFast, 
                          BertForSequenceClassification)
from transformers import Trainer, TrainingArguments
from datasets import load_dataset

device = 'cuda' if cuda.is_available() else 'cpu'

In [1]:
import torch
from datasets import load_dataset
from adapters import AutoAdapterModel, AdapterTrainer, AdapterConfig
from transformers import AutoTokenizer, TrainingArguments


/home/tuvis/miniconda3/envs/hf/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:

# Load the IMDB dataset
dataset = load_dataset("imdb")

# Load pre-trained BERT model and tokenizer
model_name = "bert-base-uncased"
model = AutoAdapterModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    result = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)
    result["labels"] = examples["label"]
    return result

tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=dataset["train"].column_names)
tokenized_datasets.set_format("torch")



Map: 100%|██████████| 50000/50000 [00:12<00:00, 3946.04 examples/s]


In [17]:
# Add a new adapter
adapter_name = "imdb_sentiment"
model.add_adapter(adapter_name, config="pfeiffer")
model.add_classification_head(adapter_name, num_labels=2)
model.set_active_adapters(adapter_name)
model.train_adapter([adapter_name])

/home/tuvis/miniconda3/envs/hf/lib/python3.11/site-packages/adapters/composition.py:186: FutureWarning: Passing list objects for adapter activation is deprecated. Please use Stack or Fuse explicitly.
  warnings.warn(


In [18]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)


/home/tuvis/miniconda3/envs/hf/lib/python3.11/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [19]:
# Initialize AdapterTrainer
trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)


In [20]:
# Train the adapter
trainer.train()


  0%|          | 0/9375 [04:26<?, ?it/s]           

{'loss': 0.697, 'grad_norm': 2.010999917984009, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.0}



  0%|          | 0/9375 [04:30<?, ?it/s]           

{'loss': 0.7126, 'grad_norm': 2.0656707286834717, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.01}



  0%|          | 0/9375 [04:34<?, ?it/s]           

...


  0%|          | 0/9375 [09:04<?, ?it/s]            

{'loss': 0.2543, 'grad_norm': 3.757408618927002, 'learning_rate': 4.904225352112676e-05, 'epoch': 0.21}



  0%|          | 0/9375 [09:09<?, ?it/s]            

{'loss': 0.2636, 'grad_norm': 1.1920865774154663, 'learning_rate': 4.898591549295775e-05, 'epoch': 0.22}


KeyboardInterrupt: 

In [15]:
# Add a new adapter
adapter_name = "imdb_sentiment"
model.add_adapter(adapter_name, config="pfeiffer")

# Activate the adapter
model.set_active_adapters(adapter_name)

# Prepare the model for adapter training
model.train_adapter([adapter_name])

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
)

# Initialize AdapterTrainer
trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)


# Train the adapter
trainer.train()

/home/tuvis/miniconda3/envs/hf/lib/python3.11/site-packages/adapters/composition.py:186: FutureWarning: Passing list objects for adapter activation is deprecated. Please use Stack or Fuse explicitly.
  warnings.warn(
  0%|          | 0/9375 [00:00<?, ?it/s]

ValueError: Expected input batch_size (4096) to match target batch_size (8).

In [13]:

# Add a new adapter
adapter_name = "imdb_sentiment"
model.add_adapter(adapter_name, config="pfeiffer")

# Activate the adapter
model.set_active_adapters(adapter_name)

# Prepare the model for adapter training
model.train_adapter([adapter_name])

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
)

# Initialize AdapterTrainer
trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)


# Train the adapter
trainer.train()


ValueError: An adapter with the name 'imdb_sentiment' has already been added.

  0%|          | 0/9375 [00:00<?, ?it/s]

ValueError: Expected input batch_size (4096) to match target batch_size (8).

In [ ]:
# Train the adapter
trainer.train()

# Save the adapter
model.save_adapter("./imdb_adapter", adapter_name)

# Load the adapter (for future use)
loaded_model = AutoAdapterModel.from_pretrained(model_name)
loaded_model.load_adapter("./imdb_adapter")

In [ ]:
!pip install accelerate -U